<a href="https://colab.research.google.com/github/avtar123/Face-Recognition/blob/main/face_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data **Preprocessing**

In [ ]:
import os
import cv2
import pickle
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from moviepy.video.io.VideoFileClip import VideoFileClip


In [ ]:
crp_img_dir = "/content/drive/MyDrive/Players_face_dataset/"
crp_img_dir1 = os.listdir(crp_img_dir)

In [ ]:
player_names_dict = {}
count = 0
for player in crp_img_dir1:
    player_names_dict.update({"{}".format(count):player})
    count+=1
print(player_names_dict)

{'0': 'Ajinkya_Rahane', '1': 'bhuvneshwar_kumar', '2': 'David_Warner', '3': 'deepak_chahar', '4': 'dinesh_karthik', '5': 'Glen_Max', '6': 'hardik_pandya', '7': 'jasprit_bumrah', '8': 'KL_Rahul', '9': 'ms_dhoni', '10': 'Quinton_De_Kock', '11': 'ravindra_jadeja', '12': 'Rohit_Sharma', '13': 'shikhar_dhawan', '14': 'Shubman_Gill', '15': 'virat_kohli'}


In [ ]:
class_dict = {}
count = 0
for player_id in player_names_dict.keys():
  class_dict[player_id] = count
  count = count + 1
class_dict


{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 '10': 10,
 '11': 11,
 '12': 12,
 '13': 13,
 '14': 14,
 '15': 15}

In [ ]:
def hog_feature(img):
  win_size = (64,64)
  block_size = (16,16)
  block_stride = (8,8)
  cell_size = (8,8)
  num_bins = 9
  hog = cv2.HOGDescriptor(win_size,block_size,block_stride,cell_size,num_bins)
  hog_features = hog.compute(img)
  #hog_features = hog_features.flatten()
  hog_features = np.array(hog_features).flatten()
  #hog_img = hog_features.reshape(-1,num_bins)
  scaler = StandardScaler()
  hog_features_normalized = scaler.fit_transform(hog_features.reshape(-1,1))
  return hog_features_normalized

In [ ]:
x_train, y_train, x_test, y_test = [], [], [], []
min_len = 500
for player_id, player_name in player_names_dict.items():
    player_img_dir = os.path.join(crp_img_dir,player_name)
    player_face_img_dir = os.listdir(player_img_dir)
    #print(player_img_dir)
    #print(player_face_img_dir)
    len_dir = len(player_face_img_dir)
    if min_len>len_dir:
      min_len = len_dir
print(min_len)
for player_id, player_name in player_names_dict.items():
  #print(celebrity_name,player_name)
  player_img_dir = os.path.join(crp_img_dir,player_name)
  player_face_img_dir = os.listdir(player_img_dir)
  #print(player_img_dir)
  #print(player_face_img_dir)
  split_counter = 0
  #for player_img in player_face_img_dir[:min_len]:
  for player_img in player_face_img_dir:
    #print(player_img)
    img = cv2.imread(player_img_dir+"/"+player_img)
    #print(img)
    gray_img = cv2.cvtColor( img,cv2.COLOR_RGB2GRAY )
    #print(gray_img)
    cv2.imwrite("img.png",gray_img)
    if gray_img is not None:
      scaled_img = cv2.resize(gray_img, (64,64))
      #cv2.imwrite("img.png",scaled_img)
      #img_har = w2d(gray_img,'db1',5)
      #scaled_img_har = cv2.resize(img_har, (32, 32))
      #combined_img = np.vstack((scalled_img.reshape(32*32*3,1),scalled_img_har.reshape(32*32,1)))
      hog_img = hog_feature(scaled_img)
      #cv2.imwrite("img_1.png",hog_img)
      if split_counter<5:
        x_test.append(hog_img)
        y_test.append(class_dict[player_id])
      else:
        x_train.append(hog_img)
        y_train.append(class_dict[player_id])
      #break
    split_counter+=1

13


In [ ]:
type(x_train[0])

numpy.ndarray

In [ ]:
x_train = np.array(x_train).reshape(len(x_train),-1)
x_test = np.array(x_test).reshape(len(x_test),-1)

In [ ]:
print(x_train[0].shape)

(1764,)


In [ ]:
type(y_train)

list

In [ ]:
print(len(x_train),len(y_train),len(x_test),len(y_test))

940 940 80 80


# **Training**

In [ ]:
pipe = Pipeline([('svc', SVC(kernel = 'poly', C = 10))])
#model = svm.SVC(kernel = 'rbf', C = 10)
pipe.fit(x_train, y_train)
pipe.score(x_test, y_test)

0.45

In [ ]:
pickle.dump(model, open("/content/drive/MyDrive/player_recognition_model.sav", 'wb'))

# **Prediction** ***on*** **image**

In [ ]:
pre_img = cv2.imread("/content/ms_dhoni_1331.png")
pre_gray_img = cv2.cvtColor( pre_img,cv2.COLOR_RGB2GRAY )
scaled_pre_gray_img = cv2.resize(pre_gray_img, (64,64))

In [ ]:
scaled_pre_gray_img.shape

(64, 64)

In [ ]:
pre_hog_img = hog_feature(scaled_pre_gray_img)

In [ ]:
pre_hog_img

array([[ 2.0510895],
       [-1.0184659],
       [-0.6462854],
       ...,
       [-1.1925026],
       [-1.06899  ],
       [-1.2671126]], dtype=float32)

In [ ]:
pre_hog_img = np.array(pre_hog_img).reshape(1,-1)

In [ ]:
result = pipe.predict(pre_hog_img)

In [ ]:
player_names_dict["{}".format(result[0])]

'ms_dhoni'

# **Prediction** **on** **a** **video**

In [ ]:
vid = VideoFileClip(/content/drive/MyDrive/Video_data/Best of Dhoni Mahi .mp4)
model = hubconf._create(/content/drive/MyDrive/best.pt)
player_name = os.listdir(images_path)
player_names_dict = player_dict(player_name)
for duration in range(0,int(vid.duration)):

    frame = vid.get_frame(t = duration)
    results = np.array(model(frame).pred[0].cpu())
    results1= np.array(results[np.isin(results[:,5],np.array([0]))])
    if len(results)>0:
        for i in results:
            crop_face = frame[int(i[1]):int(i[3]),int(i[0]):int(i[2])]
            print(crop_face)
            img_rgb = cv2.cvtColor(crop_face,cv2.COLOR_BGR2RGB)
            gray_img = cv2.cvtColor( img_rgb,cv2.COLOR_RGB2GRAY )
            scaled_gray_img = cv2.resize(gray_img, (64,64))
            hog_img = hog_feature(scaled_gray_img)
            final_img = np.array(hog_img).reshape(1,-1)
            Player_name = player_recog(final_img,face_recogniton_model,player_names_dict)
            print(Player_name)

In [ ]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto',probability=True),
        'params' : {
            'svc__C': [1,10,100,1000],
            'svc__kernel': ['rbf','linear','poly','sigmoid']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'randomforestclassifier__n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='lbfgs',multi_class='auto'),
        'params': {
            'logisticregression__C': [1,5,10]
        }
    }
}


In [ ]:
model_params

{'svm': {'model': SVC(gamma='auto', probability=True),
  'params': {'svc__C': [1, 10, 100, 1000],
   'svc__kernel': ['rbf', 'linear', 'poly', 'sigmoid']}},
 'random_forest': {'model': RandomForestClassifier(),
  'params': {'randomforestclassifier__n_estimators': [1, 5, 10]}},
 'logistic_regression': {'model': LogisticRegression(),
  'params': {'logisticregression__C': [1, 5, 10]}}}

In [ ]:
scores = []
best_estimators = {}
for algo, mp in model_params.items():
    pipe = make_pipeline(mp['model'])
    clf =  GridSearchCV(pipe, mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train)
    scores.append({
        'model': algo,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    best_estimators[algo] = clf.best_estimator_

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df